In [23]:
import arff
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from src.medidas_desempeno_biclase import calcula_medidas_biclase
from src.visualiza_svm import visualiza_svm

In [28]:
with open('data/2year.arff') as f:
    dataset = arff.load(f)

df = pd.DataFrame(dataset['data'], columns=[attr[0] for attr in dataset['attributes']])
df = df.dropna()
print(f'df.shape: {df.shape}')
df.head()

df.shape: (4088, 65)


,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class
0,0.202350,0.46500,0.240380,1.5171,-14.547,0.51069,0.253660,0.91816,1.1519,0.42695,...,0.131840,0.473950,0.86816,0.00024,8.5487,5.1655,107.740,3.3879,5.34400,0
5,0.287840,0.55225,0.316340,1.7033,38.803,0.39012,0.287840,0.80761,1.2920,0.44600,...,0.225990,0.645370,0.77401,0.22969,7.8151,3.1534,110.830,3.2933,6.33460,0
9,0.009901,0.71174,0.065186,1.1485,-62.297,0.00000,0.013459,0.40500,1.1738,0.28826,...,0.203090,0.034347,0.83056,0.72047,7.7605,4.1253,136.540,2.6732,2.36800,0
13,0.179490,0.42473,0.552350,2.4516,42.408,0.55343,0.223930,1.35440,1.1023,0.57527,...,0.092802,0.312010,0.90720,0.07688,9.3788,4.9068,53.734,6.7927,38.49200,0
14,0.124600,0.20223,0.169850,3.1563,92.043,0.42676,0.147910,3.94480,1.2747,0.79777,...,0.215480,0.156190,0.78452,0.15476,9.7697,3.1420,51.824,7.0431,0.73834,0


In [29]:
print(df['class'].value_counts())
mayor = df['class'].value_counts().max()
menor = df['class'].value_counts().min()
print(f'IR: {np.round(mayor/menor, 2)}')

class
0    4015
1      73
Name: count, dtype: int64
IR: 55.0


### Particion

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('class', axis=1),
    df['class'],
    test_size=0.2,
    stratify=df['class'],
    random_state=42)

encoder_df = LabelEncoder()
y_train = encoder_df.fit_transform(y_train)
y_test = encoder_df.transform(y_test)

### Entrenar SVM

In [27]:
svm_model = svm.SVC()
cv_vals = [5, 10]
resultados_svm_original = []

params = {'kernel' : ['linear'],
    'C': [3, 5],
    'gamma': [0.0002, 0.0003]}

medidas = {
        'balanced_accuracy': 'balanced_accuracy'}

for cval in cv_vals:    
    grid = GridSearchCV(
        estimator = svm_model,
        param_grid = params,
        cv = cval,
        scoring = medidas,
        refit = 'balanced_accuracy',
        n_jobs = -1)

    grid.fit(X_train, y_train)
    resultados_svm_original.append({
        'cv': cval,
        'best_score': grid.best_score_,
        'best_params': grid.best_params_, 
        'medidas': grid.cv_results_})

svm_model = grid.best_estimator_
    
print(f"Modelo SVM con kernel={svm_model.kernel} y C={svm_model.C}")

KeyboardInterrupt: 